# Encoding with Altair

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shinchu/dataviz-notebooks/blob/main/week_3/encoding-with-altair.ipynb)

Altairを使って、データから可視化への変換をもう少し練習してみましょう。

In [ ]:
# ライブラリのインストール（最初に実行する時のみ）

!pip install altair
!pip install vega_datasets

In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data as vega_data

## Gapminderのデータの読み込み

1955年から2005年までの期間における、いくつかの国の健康と人口データを可視化してみましょう。このデータはGapminder Foundationによって収集され、ハンス・ロスリングのTEDトークで共有されたものです。

まず、vega-datasetsコレクションからデータセットをPandasのデータフレームにロードします。

In [ ]:
data = vega_data.gapminder()

データを確認します。

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head()

各国（`country`）・各年（`year`, 5年間隔）ごとに、女性一人当たりの子供の数（`fertility`）、平均寿命（`life_expect`）、総人口（`pop`）を測定しています。これがデータ変数です。

また、クラスタのフィールドには、整数が表示されています。これは何を表しているのでしょうか？データを可視化しながら、これを明らかにしていきましょう。

さらに、2000年の値だけに絞り込んで、より小さなデータフレームを作成します。

In [ ]:
data2000 = data[data["year"] == 2000]

In [ ]:
data2000.head()

## データ変数の性質

Altairではテキストを直接扱うことはできません。また、間隔尺度と比例尺度を量的尺度としてまとめて扱っています。

そのため、ここではデータ変数の性質のうち、名義尺度、順序尺度、量的尺度（間隔尺度、比例尺度）について見ていきます。

### 名義尺度（Nominal, N）

名義尺度のデータ変数は、はカテゴリー名で構成されます。

値Aと値Bは同じか違うか（A = B）というような、値の等質性を比較することができます。上のデータセットでは、`country`は名義尺度です。

位置、色相（青、赤、緑など）、形状を見れば、値が同じか異なるかがすぐに分かります。しかし、大きさを使用して名義尺度を変換すると、存在しない値間の順位や大きさの違いを示唆し、誤解を招く恐れがあります。

### 順位尺度（Ordinal, O）

順序データは、特定の順序を持つ値で構成されます。

値Aは値Bの前か後か?（A < B）というような比較をすることができます。上のデータセットでは、5年間隔の`year`を順序尺度として扱うことができます。

順序尺度のデータ変数を可視化する場合は、順序の感覚に合わせる必要があります。位置、大きさ、明るさなどが適切で、色相（知覚的に順序がないか直感的には分からない）はあまり適切ではありません。

### 量的尺度（Quantitative, Q）

量的尺度では、数値の違いを測定することができます。量的尺度には、2種類あります。

間隔尺度では、点間の距離（区間）を測定することができます。値Bから値Aまでの距離は何メートルk？（A - B）のような計算をすることができます。

比例尺度では、ゼロに意味があるので、比率を計算することができます。値Aは値Bの何％か？（A / B）のような計算が可能です。

上のデータセットでは、`year`は間隔尺度（yearの値「0」は一つの目安であり、量が存在しないことを意味しない）、`fertility`と`life_expect`は比例尺度です。

量的尺度は、位置、サイズ、明るさなどを使用して可視化することができます。ベースラインがゼロの軸は、比例尺度の比較には必須ですが、間隔尺度の比較では省略することが可能です。

### 補足

これらのデータ型は相互に排他的ではなく、階層を形成しています。量的尺度は順序尺度を含み、順序尺度は名義尺度を含みます。

そのため、変数は1つの性質のみを持っているわけではありません。変数が数値で表現されていても、名義尺度や順序尺度として扱うことができます。例えば、年齢（10歳、20歳など）は、名義尺度（未成年、成人）、順序尺度（年ごとにグループ化）、定量尺度として扱うことができます。

次に、これらのデータ変数をどのように可視化するかを見てみましょう。

In [ ]:
# 授業用：PythonのWarning（警告）を非表示にする設定
import warnings
warnings.filterwarnings("ignore")

## 視覚変数

Altairでは、視覚記号をmarkといいます。また、視覚変数はencoding channelに含まれます。

主な視覚変数は次の通りです。

* `x`: x軸の位置
* `y`: y軸の位置
* `size`: 視覚記号（mark）の大きさ。視覚記号によって、面積または長さに対応する
* `color`: 色
* `opacity`: 不透明度。0（透明）–1（不透明）の値をとる
* `shape`: 視覚記号の形

その他、視覚変数ではなく、（次回勉強する）デザイン原則もencoding channelに含まれます。

* `tooltip`: 視覚記号の上にマウスを置いたときに表示される内容
* `order`: 視覚記号の描画順序
* `column`: 複数の可視化を左右に描画
* `row`: 複数の可視化を上下に描画

### X

`x`は、視覚記号のx座標を設定します。軸とタイトルはデフォルトで表示されます。量的尺度を選択すると、軸のスケールが連続した直線になります。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q')
)

### Y

`y`は、視覚記号のy座標を設定します。ここでは、順序尺度のデータ（O）を使用しています。その結果、値ごとの離散軸が得られました。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:O')
)

【演習】 OをQにしたらどうなるでしょうか？確認してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:O'),
    alt.Y('cluster:O')
)

`cluster`のかわりに`life_expect`を量的尺度として追加すると、両軸が線形スケールの散布図になります。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q')
)

デフォルトでは、線形定量尺度の軸は、比例尺度のデータを比較するための適切なベースラインを確保するためにゼロを含んでいます。

しかし、場合によっては、ゼロの基準線は無意味であったり、区間比較に焦点を当てたいことがあります。ゼロを自動的に含まないようにするには、`scale`を指定します。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q', scale=alt.Scale(zero=False)),
    alt.Y('life_expect:Q', scale=alt.Scale(zero=False))
)

これで、軸にゼロが含まれなくなりました。軸の端は5や10の倍数のようなniceな数字に自動的に調整されるため、いくらかの余白が残っています

【演習】 上記のscale属性に`nice=False`を追加するとどうなるでしょうか？試してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q', scale=alt.Scale(zero=False, nice=False)),
    alt.Y('life_expect:Q', scale=alt.Scale(zero=False, nice=False))
)

### Size

`size`は、視覚記号の大きさを設定します。この変数の意味は、視覚記号の種類によって異なります。点（`point`）の場合は、点の面積に対応します。

`size`に人口（`pop`）を指定することによって、散布図を拡張してみましょう。その結果、このグラフには点の大きさを解釈するための凡例（legend）も含まれるようになりました。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q')
)

場合によっては、デフォルトのサイズ範囲ではうまくいかないことがあります。サイズの範囲を指定するには、scale属性の`range`パラメータに、最小と最大のサイズを示す配列を渡します。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0, 1000]))
)

### Color and Opacity

`color`は、視覚記号の色を設定します。名義尺度はカテゴリカルな配色になりますが、順序尺度と量的尺度は連続的なカラーグラデーションが使われます。

ここでは、`cluster`変数を`color`を使って表現してみます。

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

【演習】円を塗りつぶしたいときは、`mark_point`に対して`filled=True`と指定すればOKです。試してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N')
)

重ねて表示することに対応するために、塗りつぶしはデフォルトでは半透明になっています。`opacity`を指定することで、透明度を調整できます。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.9)
)

### Shape

`shape`は、視覚記号の形を指定します。この視覚変数は、視覚記号pointに対してのみ有効です。また、名義尺度にのみ使うことができます。

`cluster`に対して、色だけではなく形も指定してみます。同じデータ変数に対して複数の視覚変数を使用することは、冗長エンコーディングとして知られています。が、色と形やパターンを併用することは、アクセシビリティを確保する上で重要です（色覚障害などに対応）。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Shape('cluster:N')
)

### Tooltips and Ordering

ここまで図を作ってきましたが、どの国がどの点に対応するのかがまだ分かりません。国名を表示し、データ探索ができるようにしましょう。



In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country')
)

これで、マウスカーソルをあてると、国名が表示されるようになりました。しかし、重なっている部分に表示できない国名があることが分かります。

描画順序を調整してみましょう。`pop`を降順に並べ、小さい円が大きい円より後に描画されるようにします。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
)

これで、例えばIndiaに隠れていたBangladeshを表示できるようになりました。

また、Tooltipには複数の値を表示することもできます。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Order('pop:Q', sort='descending'),
    tooltip = [
        alt.Tooltip('country:N'),
        alt.Tooltip('fertility:Q'),
        alt.Tooltip('life_expect:Q')
    ]   
)

### Column and Row Facets

複数の可視化を配置する方法として、行や列からなるサブプロットを並べることができます。

ここでは、`cluster`ごとに分割して並べてみましょう。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000])),
    alt.Color('cluster:N'),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
)

上の可視化は画面に収まらないので、複数の図を一度に比較するのが難しいです。図の大きさを調整してみましょう。

また、`cluster`はすでに列ごとに表現されているので、`legend=None`として凡例を削除しましょう。

更に、`pop`の凡例を図の下部に持ってきてスペースを節約します。

In [ ]:
alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]),
             legend=alt.Legend(orient='bottom', titleOrient='left')),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Column('cluster:N')
).properties(width=135, height=135)

【演習】それでは、上の図を行ごとに表示するよう書き直してみたらどうなるでしょう？試してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data2000).mark_point(filled=True).encode(
    alt.X('fertility:Q'),
    alt.Y('life_expect:Q'),
    alt.Size('pop:Q', scale=alt.Scale(range=[0,1000]),
             legend=alt.Legend(orient='bottom', titleOrient='left')),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending'),
    alt.Row('cluster:N')
).properties(width=135, height=135)

### Interaction

データ探索に便利な機能として、スライダーを追加することができます。

全体のデータフレームを使って、`year`をスライダーに割り当ててみましょう。

年を経るごとに、`fertility`と`life_expect`はどのように変化しているでしょうか？また、異なる国の相対的な変化はどうでしょうか？

In [ ]:
select_year = alt.selection_single(
    name='select', fields=['year'], value=[{'year': 1955}],
    bind=alt.binding_range(min=1955, max=2005, step=5)
)

alt.Chart(data).mark_point(filled=True).encode(
    alt.X('fertility:Q', scale=alt.Scale(domain=[0,9])),
    alt.Y('life_expect:Q', scale=alt.Scale(domain=[0,90])),
    alt.Size('pop:Q', scale=alt.Scale(domain=[0, 1200000000], range=[0,1000])),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
).add_selection(select_year).transform_filter(select_year)

**エラーが出たら、下のコードを使ってください。**

In [ ]:
select_year = alt.selection_single(
    name='select', fields=['year'], init={'year': 1955},
    bind=alt.binding_range(min=1955, max=2005, step=5)
)

alt.Chart(data).mark_point(filled=True).encode(
    alt.X('fertility:Q', scale=alt.Scale(domain=[0,9])),
    alt.Y('life_expect:Q', scale=alt.Scale(domain=[0,90])),
    alt.Size('pop:Q', scale=alt.Scale(domain=[0, 1200000000], range=[0,1000])),
    alt.Color('cluster:N', legend=None),
    alt.OpacityValue(0.5),
    alt.Tooltip('country:N'),
    alt.Order('pop:Q', sort='descending')
).add_selection(select_year).transform_filter(select_year)

## 視覚記号

ここまで、視覚変数の勉強のため、視覚記号として`mark_point`のみを使ってきました。

他に使用できる視覚記号として、下記があります。

* `mark_area()`：トップラインとベースラインで定義された塗りつぶし領域
* `mark_bar()`：長方形の棒
* `mark_circle()`：塗りつぶされた円
* `mark_line()`：線分
* `mark_point()`：形状を設定できる点
* `mark_rect()`：塗りつぶされた矩形
* `mark_rule()`：軸にかかる線分
* `mark_square()`：塗りつぶされた四角
* `mark_text()`：文字
* `mark_tick()`：水平か垂直の刻み目

### Point

In [ ]:
alt.Chart(data2000).mark_point().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

引数を与えることで、見た目を調整できます。

In [ ]:
alt.Chart(data2000).mark_point(filled=True, size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

### Circle

`mark_circle()`は、`mark_point(filled=True)`と等価です。

In [ ]:
alt.Chart(data2000).mark_circle(size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

### Square

In [ ]:
alt.Chart(data2000).mark_square(size=100).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

【演習】四角の大きさを200に指定してみましょう

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data2000).mark_square(size=200).encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

### Tick

`mark_tick()`は、一次元の分布を比較するのに便利です。

In [ ]:
alt.Chart(data2000).mark_tick().encode(
    alt.X('fertility:Q'),
    alt.Y('cluster:N'),
    alt.Shape('cluster:N')
)

### Bar

`mark_bar()`は、位置、幅、高さを持つ矩形を描画します。

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('country:N'),
    alt.Y('pop:Q')
)

棒を積み重ねることもできます。

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('cluster:N'),
    alt.Y('pop:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
)

デフォルトでは0から矩形を描画しますが、次の例のように始点（`X`）と終点（`X2`）を指定することもできます。

ここでは、`cluster`ごとに、平均寿命の最小値と最大値を始点と終点に指定して、棒グラフを描画してみます。

In [ ]:
alt.Chart(data2000).mark_bar().encode(
    alt.X('min(life_expect):Q'),
    alt.X2('max(life_expect):Q'),
    alt.Y('cluster:N')
)

### Line

`mark_line()`は、点をつなげる線分を描画します。これによって、例えば変化の度合いがわかります。

ここでは、国ごとの`fertility`の経年変化を見てみましょう。

In [ ]:
alt.Chart(data).mark_line().encode(
    alt.X('year:O'),
    alt.Y('fertility:Q'),
    alt.Color('country:N')
).properties(
    width=400
)

国が多すぎて凡例が途中で途切れていることがわかります。

【演習】凡例を消して、かわりにTooltipsに国名を表示してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(data).mark_line().encode(
    alt.X('year:O'),
    alt.Y('fertility:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)

`strokeWidth`で線分の太さを、`opacity`で透明度を指定することができます。

また、`interpolate`で線分をなめらかにするための補間方式を指定することができます。

In [ ]:
alt.Chart(data).mark_line(
    strokeWidth=3,
    opacity=0.5,
    interpolate='monotone'
).encode(
    alt.X('year:O'),
    alt.Y('fertility:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width=400
)

`mark_line()`は、2つの状態間の比較に使うこともできます（スロープグラフ）。

ここでは、国ごとに1955年と2005年の`fertility`を比較してみましょう。

In [ ]:
dataTime = data.loc[(data['year'] == 1955) | (data['year'] == 2005)]

alt.Chart(dataTime).mark_line(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width={"step": 100} # adjust the step parameter
)

【演習】引数`step`を調整して、値の変化が可視化とそれが与える印象にどのような変化をもたらすのかを観察してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

dataTime = data.loc[(data['year'] == 1955) | (data['year'] == 2005)]

alt.Chart(dataTime).mark_line(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N', legend=None),
    alt.Tooltip('country:N')
).properties(
    width={"step": 400} # adjust the step parameter
)

### Area

`mark_area()`は、折れ線と棒を組み合わせたようなもので、変化と大きさを同時に表現します。

アメリカの人口の経年変化を見てみましょう。

In [ ]:
dataUS = data.loc[data['country'] == 'United States']

alt.Chart(dataUS).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q')
)

棒と同様に、積み重ねることができます。

In [ ]:
dataNA = data.loc[
    (data['country'] == 'United States') |
    (data['country'] == 'Canada') |
    (data['country'] == 'Mexico')
]

alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q'),
    alt.Color('country:N')
)

デフォルトでは0から積み重ねますが、軸の中心から積み重ねたり、比率によって積み重ねることができます。

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack='center'),
    alt.Color('country:N')
)

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack='normalize'),
    alt.Color('country:N')
)

また、積み重ねるのではなく、重ね合わせることもできます。この時、不透明度を指定することで重なった層を確認できます。

In [ ]:
alt.Chart(dataNA).mark_area(opacity=0.5).encode(
    alt.X('year:O'),
    alt.Y('pop:Q', stack=None),
    alt.Color('country:N')
)

また、棒と同様に、始点と終点を指定することができます。

始点と終点にそれぞれ、北米の国の`fertility`の最小値と最大値を指定してみましょう。

In [ ]:
alt.Chart(dataNA).mark_area().encode(
    alt.X('year:O'),
    alt.Y('min(fertility):Q'),
    alt.Y2('max(fertility):Q')
).properties(
    width={"step": 40}
)

差が縮まっていることがわかりますね。

【演習】上の図のx軸とy軸を入れ替えることで、図を転置できます。試してみましょう。

In [ ]:
# your code goes here


In [ ]:
# answer

alt.Chart(dataNA).mark_area().encode(
    alt.X('min(fertility):Q'),
    alt.X2('max(fertility):Q'),
    alt.Y('year:O')
).properties(
    width={"step": 40}
)

---

これで、Altairによるエンコーディングの練習はひととおり終了です。

もっと高度な使い方については、いつもの通り、公式ドキュメントを見てみましょう。

Altair: https://altair-viz.github.io/